In [ ]:
from rw_ve_plot import *
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
plt.style.use(r"./RW_visualization.mplstyle")


In [16]:
# import warnings filter
# from pandas.core.common import SettingWithCopyWarning
import warnings
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pd.options.mode.chained_assignment = None 

In [ ]:
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('vaccine_details.xlsx')
# vaccine_detail = pd.read_excel('./Vaccine/RW_vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Drop rows if efficacy does not exist
vaccine_detail = vaccine_detail[vaccine_detail['efficacy_in_%'] != 'X']

# Remove all vaccine
vaccine_detail = vaccine_detail[vaccine_detail.vaccine!='All vaccine']

In [ ]:
plot_vaccine_efficacy(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_variants_group(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_ave_group(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_ave_group_for_variants(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_variants_and_ave_group(vaccine_detail, save_figure=False)

Average vaccine efficacy

In [ ]:
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('vaccine_details.xlsx')
# vaccine_detail = pd.read_excel('./Vaccine/RW_vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Drop rows if efficacy does not exist
vaccine_detail = vaccine_detail[vaccine_detail['efficacy_in_%'] != 'X']

plot_average_vaccine_efficacy(vaccine_detail, save_figure=False)

Use my reproduced vaccine efficacy

In [24]:
# Run vaccine_efficacy_CIs.ipynb to update the data
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('rw_vaccine_details.xlsx')
# vaccine_detail = pd.read_excel('./Vaccine/RW_vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Remove all vaccine
vaccine_detail = vaccine_detail[vaccine_detail.vaccine!='All vaccine']

In [25]:
# Replace the vaccine efficacy and Ci by my estimations
vaccine_detail['efficacy_in_%'] = vaccine_detail['rw_efficacy_in_%']
vaccine_detail['lower'] = vaccine_detail['rw_lower']
vaccine_detail['upper'] = vaccine_detail['rw_upper']

In [26]:
# Drop empty vaccine efficacy
vaccine_detail = vaccine_detail[~np.isnan(vaccine_detail['efficacy_in_%'])]

In [ ]:
vaccine_detail

In [ ]:
plot_vaccine_efficacy_ave_group_for_original(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_ave_group_for_variants(vaccine_detail, save_figure=False)